In [27]:
import pandas as pd
import numpy as np
import aocd
import timeit
import os
import math
from aocd.models import Puzzle
os.environ['AOC_SESSION'] = '53616c7465645f5fda624994231306c055524663b055568600cbd7a8e885d012db0c35ac4c420439e127563fce396857'
import matplotlib.pyplot as plt
puzzle = Puzzle(year=2019, day=10)

In [100]:
print(puzzle.input_data)

....#...####.#.#...........#........
#####..#.#.#......#####...#.#...#...
##.##..#.#.#.....#.....##.#.#..#....
...#..#...#.##........#..#.......#.#
#...##...###...###..#...#.....#.....
##.......#.....#.........#.#....#.#.
..#...#.##.##.....#....##..#......#.
..###..##..#..#...#......##...#....#
##..##.....#...#.#...#......#.#.#..#
...###....#..#.#......#...#.......#.
#....#...##.......#..#.......#..#...
#...........#.....#.....#.#...#.##.#
###..#....####..#.###...#....#..#...
##....#.#..#.#......##.......#....#.
..#.#....#.#.#..#...#.##.##..#......
...#.....#......#.#.#.##.....#..###.
..#.#.###.......#..#.#....##.....#..
.#.#.#...#..#.#..##.#..........#...#
.....#.#.#...#..#..#...###.#...#.#..
#..#..#.....#.##..##...##.#.....#...
....##....#.##...#..........#.##....
...#....###.#...##........##.##..##.
#..#....#......#......###...........
##...#..#.##.##..##....#..#..##..#.#
.#....#..##.....#.#............##...
.###.........#....#.##.#..#.#..#.#..
#...#..#...#.#.#.....#....#......###
#

In [15]:
test_string = """.#..#
.....
#####
....#
...##"""

In [44]:
def get_asteroid_positions(input_string):
    
    rows = input_string.split('\n')
    asteroid_positions = []
    
    for y, row in enumerate(rows):
        for x, mark in enumerate(row):
            if mark == '#':
                asteroid_positions.append((x,y))
    return asteroid_positions

In [45]:
asteroids = get_asteroid_positions(test_string)

In [120]:
def count_visible(asteroids, station):
    
    lines_of_sight = set()
    
    for x,y in asteroids:
        
        dx = x - station[0]
        dy = y - station[1]
        
        norm = np.sqrt(dx**2 + dy**2)
        gcd = math.gcd(dx, dy)
        
        if norm == 0:
            pass
        else:
            lines_of_sight.add((dx / gcd, dy / gcd))
        
    return len(lines_of_sight)

def get_best_station(asteroids):
    
    results = [(count_visible(asteroids, station), station) for station in asteroids]
    
    return max(results, key = lambda result_pair: result_pair[0])


In [121]:
get_best_station(asteroids)

(8, (3, 4))

In [122]:
puzzle_asteroids = get_asteroid_positions(puzzle.input_data)

In [123]:
get_best_station(puzzle_asteroids)

(329, (25, 31))

In [93]:
count_visible(asteroids, asteroids[0])

self


{(-1, 2), (0, 1), (1, 0), (1, 1), (1, 2), (3, 2), (3, 4)}

In [5]:
# note, X is second coordinate, Y first
def make_asteroid_map(input_string):
    rows = input_string.split('\n')
    
    asteroid_map = np.zeros((len(rows[0]), len(rows)))
    
    for i, row in enumerate(rows):
        for j, mark in enumerate(row):
            if mark == '#':
                asteroid_map[i,j] = 1
    return asteroid_map

In [112]:
test_asteroid_positions = get_asteroid_positions(test_string)

In [113]:
test_asteroid_positions

[(1, 0),
 (4, 0),
 (0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (4, 3),
 (3, 4),
 (4, 4)]

In [16]:
test_map = make_asteroid_map(test_string)

In [109]:
def is_colinear_triple(A, B, C):
    
    return (A[0] * (B[1] - C[1]) + B[0] * (C[1] - A[1]) + C[0] *(A[1] - B[1])) == 0

def C_between_A_and_B(A, B, C):
    
    return is_colinear_triple(A, B, C) and (abs(A[0]-C[0]) < abs(A[0]-B[0]))

def check_if_B_is_visible_from_A(A, B, asteroid_positions):
    
    for C in asteroid_positions:
        if check_if_C_blocks_vision_of_A_on_B(A, B, C) and C != B and C != A:
            return False
        
    return True
    
def calculate_number_of_visible_asteroids(A, asteroid_positions):
    visible = 0    
    
    for B in asteroid_positions:
        if check_if_B_is_visible_from_A(A, B, asteroid_positions) and B != A:
            visible += 1
    
    return visible
        
    

In [9]:
def C_between_A_and_B(A, B,C ):
    
    v1 = np.array(B) - np.array(A)
    v2 = np.array(C) - np.array(A)
    
    cross_product = np.cross(v1,v2)
    if abs(cross_product) > 0:
        return False
    
    inner_product =  v1 @ v2 #np.inner(v1, v2)
    if inner_product < 0:
        return False
    
    if inner_product > v1 @ v1:
        return False
    
    return True

def A_sees_B(A, B, asteroid_positions):
    
    for C in asteroid_positions:
        if C not in [A, B]:
            if C_between_A_and_B(A, B, C):
                return False
    
    return True

def asteroids_visible_from_A(A, asteroid_positions):
    visible = 0
    
    for B in asteroid_positions:
        if B is not A:
            if A_sees_B(A, B, asteroid_positions):
                visible += 1
    
    return visible

def make_number_visible_map(asteroid_map):

    asteroid_positions = list(zip(*np.nonzero(asteroid_map)))
    
    numbers_visible = np.zeros(asteroid_map.shape)

    for asteroid_position in asteroid_positions:
        numbers_visible[asteroid_position] = asteroids_visible_from_A(asteroid_position, asteroid_positions)
        
    return numbers_visible

def get_best_asteroid(asteroid_map):
    
    numbers_visible_map = make_number_visible_map(asteroid_map)
    
    best_position = np.unravel_index(np.argmax(numbers_visible_map, axis=None), shape = numbers_visible_map.shape)
    
    highest_number_visible = numbers_visible_map[best_position]
    
    return highest_number_visible, (best_position[1], best_position[0])

In [17]:
get_best_asteroid(test_map)

(8.0, (3, 4))

In [11]:
test2 = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""

In [12]:
test2_map = make_asteroid_map(test2)

In [13]:
get_best_asteroid(test2_map)

KeyboardInterrupt: 